In [1]:
import numpy as np
import pandas as pd
from astropy.utils.data import get_pkg_data_filename
from astropy.table import Table, Column, vstack
from astropy.io import ascii
from astropy import units as u
import math
from synphot import SourceSpectrum, Observation
from synphot import units
import stsynphot as stsyn

#np.set_printoptions(threshold=np.inf)  
from pprint import pprint 


In [2]:
class Targets:
    def __init__(self,name,redshift,lum_dist,obs_flux,Ha_flux):
        self.n = name
        self.z = redshift
        self.Ld = lum_dist
        self.ObFl = obs_flux        # in FLAM units
        self.HaFl = Ha_flux         # in ergs/sec/cm^2

In [3]:
def redshift(model, r):
     
    model_zc = SourceSpectrum(sp.model, z=r, z_type='conserve_flux')
    
    return model_zc                             


def convolved_obs(target_name, model_sp_cf):
    
    filter_lambda = 0.0*u.AA 
    bp_225 = stsyn.band('wfc3,uvis2,f225w')
    bp_275 = stsyn.band('wfc3,uvis2,f275w')  
    bp_336 = stsyn.band('wfc3,uvis2,f336w')

    if target_name == '1037':
        bp = bp_225
        filter_lambda = 2341.0*u.AA
    elif target_name == '1221':
        bp = bp_275
        filter_lambda = 2715.3*u.AA 
    else:
        bp = bp_336
        filter_lambda = 3361.1*u.AA 
    
    obs_cf = Observation(model_sp_cf, bp, binset=bp.binset)
    
    return obs_cf


def compute_sfr(obs_flux, model_flux):
    
    obs_flux = obs_flux*units.FLAM      
    sfr = obs_flux/model_flux
    
    #if m==1 or m==3 or m==5:       
        #sfr = 1.0e6*sfr
    
    return sfr


def compute_ionizing_phot_cont_model(m, dir_path):
    
    df = pd.read_table(dir_path, sep='\s+', skiprows=[0], header=None, 
                           names=('Age', 'solid', 'long_dash', 'short_dash'))
    
    if m==2:    
        Qtot_1Myr = 10**(df.loc[df['Age'] == 1e6, ['solid']].values[0][0]) 
        Qtot_10Myr = 10**(df.loc[df['Age']==1e7, ['solid']].values[0][0])
        Qtot_100Myr = 10**(df.loc[df['Age']==1e8, ['solid']].values[0][0])
    
    if m==4:
        Qtot_1Myr = 10**(df.loc[df['Age'] == 1e6, ['long_dash']].values[0][0]) 
        Qtot_10Myr = 10**(df.loc[df['Age']==1e7, ['long_dash']].values[0][0])
        Qtot_100Myr = 10**(df.loc[df['Age']==1e8, ['long_dash']].values[0][0])
            
    if m==6:
        Qtot_1Myr = 10**(df.loc[df['Age'] == 1e6, ['short_dash']].values[0][0]) 
        Qtot_10Myr = 10**(df.loc[df['Age']==1e7, ['short_dash']].values[0][0])
        Qtot_100Myr = 10**(df.loc[df['Age']==1e8, ['short_dash']].values[0][0])
      
    
    return Qtot_1Myr, Qtot_10Myr, Qtot_100Myr


def compute_ionizing_phot_inst_model(m, dir_path):
    
    df = pd.read_table(dir_path, sep='\s+', skiprows=[0], header=None, 
                           names=('Age', 'solid', 'long_dash', 'short_dash'))
        
    if m==1:
        Qtot_1Myr = 10**(df.loc[df['Age']==1.01e6, ['solid']].values[0][0])
        Qtot_10Myr = 10**(df.loc[df['Age']==1.001e7, ['solid']].values[0][0])
        Qtot_100Myr = 10**(df.loc[df['Age']==1.0001e8, ['solid']].values[0][0])
        
    if m==3:
        Qtot_1Myr = 10**(df.loc[df['Age']==1.01e6, ['long_dash']].values[0][0])
        Qtot_10Myr = 10**(df.loc[df['Age']==1.001e7, ['long_dash']].values[0][0])
        Qtot_100Myr = 10**(df.loc[df['Age']==1.0001e8, ['long_dash']].values[0][0])
        
    if m==5:
        Qtot_1Myr = 10**(df.loc[df['Age']==1.01e6, ['short_dash']].values[0][0])
        Qtot_10Myr = 10**(df.loc[df['Age']==1.001e7, ['short_dash']].values[0][0])
        Qtot_100Myr = 10**(df.loc[df['Age']==1.0001e8, ['short_dash']].values[0][0])
        
    
    return Qtot_1Myr, Qtot_10Myr, Qtot_100Myr


def compute_ionizing_phot_obs(Ha_flux, Lum_dist):
    
    Qtot = 0.0
    L_Ha = Ha_flux * (4 * math.pi * (Lum_dist * 3.08e24)**2)     # ergs/sec
        
    c = 3.0e10                   # in cm/sec
    h = 6.626e-27                # in cgs units:   cm^2 g s^-1
    nu_Ha = c/6.56e-5            # wavelength in cm
    
    Qtot = (2.2*L_Ha)/(h*nu_Ha)     # units:   sec^-1
    
    return Qtot
    

In [4]:
path = '/Users/orion/phd_research/stellar_mod/'

inputdir_cont = path+'s99_models/continuous/'
inputdir_inst = path+'s99_models/instantaneous/'
iphot_model_cont = path+'ioniz_phot_calc/912_cont_c.dat'
iphot_model_inst = path+'ioniz_phot_calc/912_inst_c.dat'


#(Gal-corrected only)
s1 = Targets('1025+390',0.361,1940.5,8.32e-18,1.53e-15)
s2 = Targets('1037+30',0.091,417.2,1.11e-16,5.06e-15)
s3 = Targets('1128+455',0.404,2217.0,6.61e-18,2.58e-15)
s4 = Targets('1201+394',0.445,2488.2,1.43e-18,0.00)            # no Ha-flux available
s5 = Targets('1203+645',0.371,2004.1,7.73e-18,1.00e-14)
s6 = Targets('1221-423',0.171,826.8,2.45e-16,1.69e-15)

sources = [s1, s2, s3, s4, s5, s6]


In [6]:
########################################### Continuous star formation 


for m in range(2,8,2):          

    df = pd.read_table(inputdir_cont+'fig'+str(m)+'c.dat', sep='\s+', skiprows=[0,1], usecols=[0,1,10,28], header=0)
    column_header = df.columns.values.tolist()
    print('\nModel: fig'+str(m)+'c.dat')  
    #print(df.columns.values.tolist())
    
    targets=[]
    modelQtot_1Myr, modelQtot_10Myr, modelQtot_100Myr = compute_ionizing_phot_cont_model(m, iphot_model_cont)

    for s in sources:
        
        targets.append(s.n)
        
        Qtot_obs = 0.0
        sfr = np.zeros(3, dtype=np.float32)
        sfr_Qtot = np.zeros(3, dtype=np.float32)
        
        Qtot_obs = compute_ionizing_phot_obs(s.HaFl, s.Ld)
        
        for i in range(1,len(column_header)):      

            d = s.Ld * 3.08e24        
            df['flux_col'] = (10**df[column_header[i]]) / (4 * math.pi * d**2)           
            df.to_csv('buffer_new.txt', columns=[column_header[0],'flux_col'], sep='\t',
                        header=False, index=False) 

            sp = SourceSpectrum.from_file('buffer_new.txt')
            model_zcf = redshift(sp,s.z)
            obs_zcf = convolved_obs(s.n, model_zcf)     
            f_lambda = obs_zcf.effstim('flam')
            sfr[i-1] = np.round(compute_sfr(s.ObFl, f_lambda), decimals=2)           # sfr in units of Msolar/yr
            
            
            if i==1:
                sfr_Qtot[i-1] = np.round(Qtot_obs/modelQtot_1Myr, 2)
            elif i==2:
                sfr_Qtot[i-1] = np.round(Qtot_obs/modelQtot_10Myr, 2)
            elif i==3:
                sfr_Qtot[i-1] = np.round(Qtot_obs/modelQtot_100Myr, 2)
        
            
            # tabulating computed values 
            if i==1:
                data1=[]
                data1.append(sfr[i-1]) 
            else:
                data1.append(sfr[i-1])

                
            if i==1:
                data2=[]
                data2.append(sfr_Qtot[i-1])
            else:
                data2.append(sfr_Qtot[i-1])
        
        
        #print(sfr, '\t', sfr_Qtot)
        #print(data1, data2)
        
    
        if s==s1:
            row1=[]
            row1.append(data1)
        else:
            row1.append(data1)

    
        if s==s1:
            row2=[]
            row2.append(data2)
        else:
            row2.append(data2)
  
  
    stacked_sfr = np.hstack((row1,row2))     
    targets_arr = np.array(targets)          
    combined = np.column_stack((targets, stacked_sfr)) 
    
    t = Table(combined, names=('sources', 'sfr_1Myr', 'sfr_10Myr', 'sfr_100Myr', 
                               'sfr_Qtot_1Myr', 'sfr_Qtot_10Myr', 'sfr_Qtot_100Myr'), 
                                     dtype=('U10','f4','f4','f4','f4','f4','f4'))     
    
    t.pprint(max_width=200)
    
    if m==2:
        cont_final_table = t
        
    else: 
        cont_final_table = vstack([cont_final_table, t])



Model: fig2c.dat
sources  sfr_1Myr sfr_10Myr sfr_100Myr sfr_Qtot_1Myr sfr_Qtot_10Myr sfr_Qtot_100Myr
-------- -------- --------- ---------- ------------- -------------- ---------------
1025+390    11.77      1.48       0.88          7.08           1.99            1.98
 1037+30     7.46      1.02       0.58          1.08            0.3             0.3
1128+455     12.0       1.5       0.89         15.57           4.37            4.35
1201+394     3.21       0.4       0.24           0.0            0.0             0.0
1203+645    11.62      1.46       0.86         49.33          13.84           13.78
1221-423    65.77      8.65       4.95          1.42            0.4             0.4

Model: fig4c.dat
sources  sfr_1Myr sfr_10Myr sfr_100Myr sfr_Qtot_1Myr sfr_Qtot_10Myr sfr_Qtot_100Myr
-------- -------- --------- ---------- ------------- -------------- ---------------
1025+390    90.56      9.46       2.75          97.0          23.48           23.22
 1037+30    60.22      6.59        1.7  

In [7]:
########################################## Instantaneous starburst


for m in range(1,7,2):

    df = pd.read_table(inputdir_inst+'fig'+str(m)+'c.dat', sep='\s+', skiprows=[0,1], usecols=[0,1,10,28], header=0)    
    column_header = df.columns.values.tolist()  
    sfr = np.zeros(len(column_header), dtype=np.float32)
    print('\nModel: fig'+str(m)+'c.dat')
    
    targets=[]
    modelQtot_1Myr, modelQtot_10Myr, modelQtot_100Myr = compute_ionizing_phot_inst_model(m, iphot_model_inst)

    for s in sources:
        
        targets.append(s.n)
        
        Qtot_obs = 0.0
        msb = np.zeros(3, dtype=np.float32)
        msb_Qtot = np.zeros(3, dtype=np.float32)
        
        Qtot_obs = compute_ionizing_phot_obs(s.HaFl, s.Ld)
        
        for i in range(1,len(column_header)):      

            d = s.Ld * 3.08e24
            df['flux_col'] = (10**df[column_header[i]]) / (4 * math.pi * d**2)
            df.to_csv('buffer_new.txt', columns=[column_header[0],'flux_col'], sep='\t',
                        header=False, index=False) 
            
            sp = SourceSpectrum.from_file('buffer_new.txt')
            model_zcf = redshift(sp,s.z)
            obs_zcf = convolved_obs(s.n, model_zcf) 
            f_lambda = obs_zcf.effstim('flam')

            # sfr => msb: mass of starburst, in case of instantaneous models
            msb[i-1] = np.round(np.log10(compute_sfr(s.ObFl, f_lambda) * 1e6), 2)         
                                                                                # msb in units of 10^6 Msolar
            

            if i==1:
                msb_Qtot[i-1] = (Qtot_obs/modelQtot_1Myr) * 1e6

            elif i==2:
                msb_Qtot[i-1] = (Qtot_obs/modelQtot_10Myr) * 1e6

            elif i==3:
                msb_Qtot[i-1] = (Qtot_obs/modelQtot_100Myr) * 1e6
                                                                            # msb_Qtot in units of 10^6 Msolar
         
            
            if i==1:
                data1=[]
                data1.append(msb[i-1]) 
            else:
                data1.append(msb[i-1])

                
            if i==1:
                data2=[]
                data2.append(msb_Qtot[i-1])
            else:
                data2.append(msb_Qtot[i-1])
                
                
        
        if s==s1:
            row1=[]
            row1.append(data1)
        else:
            row1.append(data1)

    
        if s==s1:
            row2=[]
            row2.append(data2)
        else:
            row2.append(data2)         
        

    stacked_sfr = np.hstack((row1,row2))     
    targets_arr = np.array(targets)   
    combined = np.column_stack((targets, stacked_sfr)) 
    
    t2 = Table(combined, names=('sources', 'sfr_1Myr', 'sfr_10Myr', 'sfr_100Myr', 
                               'sfr_Qtot_1Myr', 'sfr_Qtot_10Myr', 'sfr_Qtot_100Myr'), 
                                   dtype=('U10','f4','f4','f4','f4','f4','f4'))
    
    t2['sfr_1Myr'].info.format = '.2e'
    t2['sfr_10Myr'].info.format = '.2e'
    t2['sfr_100Myr'].info.format = '.2e'
    t2['sfr_Qtot_1Myr'].info.format = '.2e'
    t2['sfr_Qtot_10Myr'].info.format = '.2e'
    t2['sfr_Qtot_100Myr'].info.format = '.2e'
    t2.pprint(max_width=200)
    
    
    if m==1:
        inst_final_table = t2
        
    else: 
        inst_final_table = vstack([inst_final_table, t2])

        


Model: fig1c.dat
sources  sfr_1Myr sfr_10Myr sfr_100Myr sfr_Qtot_1Myr sfr_Qtot_10Myr sfr_Qtot_100Myr
-------- -------- --------- ---------- ------------- -------------- ---------------
1025+390 7.08e+00  7.64e+00   8.76e+00      7.50e+06       1.56e+09        5.52e+13
 1037+30 6.89e+00  7.51e+00   8.49e+00      1.15e+06       2.38e+08        8.44e+12
1128+455 7.09e+00  7.65e+00   8.78e+00      1.65e+07       3.42e+09        1.21e+14
1201+394 6.52e+00  7.07e+00   8.21e+00      0.00e+00       0.00e+00        0.00e+00
1203+645 7.07e+00  7.64e+00   8.76e+00      5.23e+07       1.08e+10        3.85e+14
1221-423 7.83e+00  8.43e+00   9.44e+00      1.50e+06       3.12e+08        1.11e+13

Model: fig3c.dat
sources  sfr_1Myr sfr_10Myr sfr_100Myr sfr_Qtot_1Myr sfr_Qtot_10Myr sfr_Qtot_100Myr
-------- -------- --------- ---------- ------------- -------------- ---------------
1025+390 7.98e+00  8.15e+00   8.82e+00      1.03e+08       6.59e+09        7.29e+13
 1037+30 7.80e+00  8.00e+00   8.53e+00  

In [ ]:
def latex_exp(value):
    
    #in case, value=0.0
    #mant = 0.0      
    #expo = 0.0 
    
    if value != 0.0:

        if value <= 1.00e2:
            return str(value)
        else:
            val = f'{value:8.2}'
            mant, exp = val.split('e')
            # remove leading zeros
            expo = exp[0] + exp[1:].lstrip('0')    
    
            return f'$ {mant} \\times 10^{{ {expo} }}$'

    else:
    
        return '\\nodata'


cont_final_table['sfr_1Myr'].format = latex_exp
cont_final_table['sfr_10Myr'].format = latex_exp
cont_final_table['sfr_100Myr'].format = latex_exp
cont_final_table['sfr_Qtot_1Myr'].format = latex_exp
cont_final_table['sfr_Qtot_10Myr'].format = latex_exp
cont_final_table['sfr_Qtot_100Myr'].format = latex_exp

inst_final_table['log(sfr_1Myr)'].format = latex_exp
inst_final_table['log(sfr_10Myr)'].format = latex_exp
inst_final_table['log(sfr_100Myr)'].format = latex_exp
inst_final_table['log(sfr_Qtot_1Myr)'].format = latex_exp
inst_final_table['log(sfr_Qtot_10Myr)'].format = latex_exp
inst_final_table['log(sfr_Qtot_100Myr)'].format = latex_exp


import sys
cont_final_table.write(sys.stdout, format='latex')
inst_final_table.write(sys.stdout, format='latex')
      